by險別，每年 *新,續,回,流失* 人數
*續* 用2021的續保單號核對

In [1]:
import os
import pandas as pd
import tqdm, datetime

os.getcwd()

'd:\\新安\\A&H 二周目\\Q4\\每年新舊回人數'

In [2]:
raw_data = pd.concat(
    [
        pd.read_csv(r'D:\新安\data\訂單整合資料\car_policy_full.txt', sep='|', dtype=str),
        pd.read_csv(r'D:\新安\data\訂單整合資料\fir_policy_full.txt', sep='|', dtype=str),
        pd.read_csv(r'D:\新安\data\訂單整合資料\hth_policy_full.txt', sep='|', dtype=str)
    ]
)

# columns typing:
raw_data[['birthY', 'year', 'plyAmt', 'clmCnt', 'clmAmt', 'clmRec', 'tg_year']] = raw_data[[
    'birthY', 'year', 'plyAmt', 'clmCnt', 'clmAmt', 'clmRec', 'tg_year']].astype(float)
raw_data["date"] = pd.to_datetime(raw_data["date"])
# new columns:
raw_data["cur_age"] = raw_data["year"] - raw_data["birthY"]
raw_data["cur_car_age"] = raw_data["year"] - raw_data["tg_year"]
raw_data["iply_person"] = raw_data["ipolicy"] + raw_data["id"]

In [3]:
used_data = raw_data.query("""
    date >= '2017-01-01' & \
    plyAmt >= 0 & \
    date <= '2022-03-31' & \
    cate != 'H_CV'
    """).copy()

In [9]:
used_data_ = used_data.copy()
used_data_.loc[used_data_["cate"].str.startswith("C"), "cate"] = "Car"
# used_data_.groupby(["cate", "year"])["id"].nunique()

for y in range(2017, 2021):
    prev = used_data.query(" cate=='CA_PS' & year==@y-1 ")
    this = used_data.query(" cate=='CA_PS' & year==@y ")
    print(this.query(" id.isin(@prev['id']) ")["id"].nunique())

0
494395
500601
528850


In [66]:

used_data.loc[used_data["cate"].str.startswith("C"), "cate"] = "Car"

data_holder = []
for i in tqdm.tqdm(used_data["cate"].unique()):
    print(f"cate={i}")
    for y in range(2017, 2022):
        print(f"year={y}")
        s = datetime.datetime.today()
        thisYear = used_data.query("""
                    year==@y & \
                    cate==@i
                    """)["id"].astype(str)
        prevYear = used_data.query("""
                        year==@y-1 & \
                        cate==@i
                        """)["id"].astype(str)
        long_time_ago = used_data.query("""
                    year<@y-1 & \
                    cate==@i
                    """)["id"].astype(str)
        new = thisYear[~thisYear.isin(prevYear) & ~thisYear.isin(long_time_ago)].nunique()
        trans = thisYear[thisYear.isin(prevYear)].nunique() #  & ~thisYear.isin(long_time_ago)
        prev_old = thisYear[~thisYear.isin(prevYear) & thisYear.isin(long_time_ago)].nunique()
        sink = prevYear[~prevYear.isin(thisYear)].nunique()

        data_holder.append(
            pd.Series([i, y, new, trans, prev_old, sink], index=["cate", "year", "新", "續", "回", "流失"])
            )

data_holder = pd.DataFrame(data_holder)

  0%|          | 0/5 [00:00<?, ?it/s]

cate=Car
year=2017
year=2018
year=2019
year=2020
year=2021


 20%|██        | 1/5 [00:29<01:59, 29.87s/it]

cate=F_PS
year=2017
year=2018
year=2019
year=2020
year=2021


 40%|████      | 2/5 [00:38<00:51, 17.20s/it]

...

 60%|██████    | 3/5 [00:45<00:25, 12.69s/it]

cate=H_TV
year=2017
year=2018
year=2019
year=2020
year=2021


 80%|████████  | 4/5 [00:56<00:11, 11.96s/it]

cate=H_GP
year=2017
year=2018
year=2019
year=2020
year=2021


100%|██████████| 5/5 [01:03<00:00, 12.66s/it]


In [96]:
data_holder
t = used_data.groupby("cate")["id"].nunique()
t = pd.concat([
    data_holder,
    pd.DataFrame(t).assign(year="不分年").reset_index().rename(columns={"id":"總人數"})
    ])
t.sort_values(["cate", "year"], inplace=True)
t["cate"] = t["cate"].apply(lambda x:
    {"Car": "車險", "F_PS":"住火", "H_PS":"個傷", "H_TV":"旅平", "H_GP":"團傷"}.get(x, x)
)


In [97]:
t.to_csv('../../data/人數_byYear_Cate.csv', index=False)

驗證

In [90]:
ply2020 = pd.read_csv(r'D:\新安\A&H\Car\data\2020\car_policy\policy.txt', dtype=str, sep='|')
irl = pd.read_csv(r'D:\新安\A&H\Car\data\2020\car_policy\ply_relation.txt', dtype=str, sep='|')

In [91]:
t = ply2020.merge(irl[['ipolicy', 'ilast_ply']], on='ipolicy', how='left')

In [95]:
print("保單續保率:", 1-irl["ilast_ply"].isna().mean())
print("人員續保率:", t[~t["ilast_ply"].isna()]["iassured"].nunique() / t["iassured"].nunique())


保單續保率: 0.6775670895345896
人員續保率: 0.7244585507240258


In [89]:
ply2020.query("ipolicy=='00009V7019469'")

,ipolicy,fassured,iassured,iassured_ext,ibirth,fsex,fmarriage,aassured_zip,iply_area,nbrand_abbr,...,iins_kit,mcover_limit,iassured_cntry,iapplicant_cntry,foccup,noccup,applicant_foccup,applicant_noccup,punknown_acc,nmile_ubi


In [73]:
raw_2020.head(2)

,ipolicy,fcard_class,irelative_ply,ilast_ply,inext_ply,itmp_policy,dply_begin,dply_end,ipro_year,ibrand,...,ibranch_at,fcomp_class,fcomp_class_last,qnext_year,irate_type,bzfrom,iroute_comm,qprovision,qdrunk_driving,isign
0,00009V7019469,2,NaN,NaN,NaN,00009CG020440,2020-09-22,2020-09-23,NaN,NaN,...,00,NaN,NaN,0,E,41,G02,0,0,882158
1,00009V7011419,2,NaN,NaN,NaN,00009CG011896,2020-07-19,2020-07-20,NaN,NaN,...,00,NaN,NaN,0,E,41,G02,0,0,882158
